This jupyter notebook <b>Calc_VectorAndTensorField_Iteration</b> calculates the local diffusion tensors and directed motion vectors, including the following steps:
- single-molecule trajectories are fed in
- unit transformation to a nanometre scale
- trajectories are filtered to meet a minimum step length for the furhter analysis
- single localisations are excluded (important for the evaluation of simulations)
- padding of feature spaces for later binning
- calculation of the local diffusion tensors and directed motion vectors ± consideration of static and dynamic localisation errors.

Consideration of the localisation errors can be modulated by the input parameter 'errcorr'.

Therefore, it is suitable to determine input parameters which are neccessary for the initial calculation of the local diffusion tenors and directed motion vectors. 

The notebook would be followed up by the jupyter notebook 'shortTrAn/Trajectory_analysis/Binning_Iteration.ipynb'.

The input matrices require following structure:
- .mat file
- 1st column: trajectory number
- 2nd column: frame number
- 3rd column: x-position
- 4th column: y-position

In [ ]:
import numpy as np
import os as os
import tifffile as tif
from loadmat import loadmat_file 
from remove_single_part import remove_single_part 
from padding import padding  
from create_fields import create_fields 
from calc_fields_out import calc_fields

In [ ]:
Num = 20 #number of datasets to be evaluated
input_path = '/Volumes/Reindeer/TrackingVSGAtto/Trc/Input_data/trc' #without '.mat'
resultpath = '/Volumes/Reindeer/TrackingVSGAtto/Trc/results'
time_lag = 10 #timelag ms
time_exp = 9 #time expose in ms
scale = 10 #pixelation of camera (160 nm) or pixelation of simulation mask (10 nm) 
traj_length = 1 #trajectories must to have a minimum step length of X to be processed 

px_padding = 160 #padding of X x1nm pix at the beginning and end of the x and y direction, respectively

sigma = 26 #localisation precision in nm
errcorr = 'no' #shall be error correction conducted? 'yes' or 'no'

In [ ]:
for it in range(1,Num+1):
    N = it #cell number
    path = (input_path + str(N))
    resultpath = resultpath
    
    #Imports data and applies the unit transfomation to 1 nm per pixel.
    a, b, tracs,tracs_unint16, trac_num, counts, tracs_unint16corr, trac_numcorr, dt = loadmat_file(path,scale)

    #Removal of all trajectories of an length shorter than 2 frames.
    tracs_unint16corr, trac_numcorr = remove_single_part(trac_num,counts,a,tracs_unint16corr,trac_numcorr, traj_length)

    #Padding. Creates additional space which is later used for the downscaling/binning.
    x_min, x_max, y_min, y_max, a, tracs_unint16corr, x_length, y_length, shape\
    = padding(tracs_unint16corr, a, px_padding, resultpath, N)
    #Creation of multiple feature space containing the countfield, diffusiontensor(xx,xy,yx,yy) 
    #and directed motion (x,y).
    pointfield,vectorfield_x,vectorfield_y,tensorfield_xx,tensorfield_xy,tensorfield_yx,tensorfield_yy=create_fields(shape)
    #Calculation of the derivates for each position of a trajectory based on the positional information of 
    #the input data.
    #Followed by the calculation of the directed motion and the tranfer of the pixel wise sum of the counts,  
    #diffusion coefficients and the speeds in the multilayer feature space.
    directory,tensorfield_xx,tensorfield_xy,tensorfield_yx,tensorfield_yy,vectorfield_x,vectorfield_y\
    =calc_fields(a,time_lag,sigma,time_exp,path,resultpath,N,errcorr,pointfield,tracs_unint16corr,
                 trac_numcorr,vectorfield_x,vectorfield_y,tensorfield_xx,tensorfield_xy,tensorfield_yx,tensorfield_yy)
    
    del N,path,a, b, tracs,tracs_unint16, trac_num, counts, tracs_unint16corr, trac_numcorr, dt,\
x_min, x_max, y_min, y_max, x_length, y_length, shape,pointfield,vectorfield_x,vectorfield_y,tensorfield_xx,\
tensorfield_xy,tensorfield_yx,tensorfield_yy,